Purpose: comparing different versions of a dataset

In [8]:
from sm_datasets.datasets import Dataset, ROOT_DIR, Example, FullTable
import ipywidgets as widgets

In [2]:
dataset_latest = Dataset(ROOT_DIR / "250wt").load()
dataset_v130 = Dataset(ROOT_DIR / "250wt/v130.zip").load()

In [17]:
assert len(dataset_latest) == len(dataset_v130)
diff_examples: list[tuple[Example[FullTable], Example[FullTable]]] = []
for ex, oldex in zip(dataset_latest, dataset_v130):
    assert ex.table.table.table_id == oldex.table.table.table_id
    if ex.sms != oldex.sms:
        diff_examples.append((ex, oldex))

In [20]:
from functools import partial

current_it = 0
output = widgets.Output()

def new_btn(**kwargs):
    if 'on_click' in kwargs:
        on_click = kwargs.pop('on_click')
    else:
        on_click = None
    btn = widgets.Button(**kwargs)
    if on_click is not None:
        btn.on_click(on_click)
    return btn


def nav(b, offset):
    global current_it
    if current_it + offset >= len(diff_examples) or current_it + offset < 0:
        return
    current_it += offset
    show_example()


def show_example():
    with output:
        output.clear_output()
        ex, oldex = diff_examples[current_it]
        print("Example:", ex.table.table.table_id, "(", current_it + 1, "of", len(diff_examples), ")")

        print("Gold models")
        
        for sm in ex.sms:
            sm.draw()
        print("Old models")
        for sm in oldex.sms:
            sm.draw()

display(widgets.HBox([
        new_btn(description="Previous", on_click=partial(nav, offset=-1)),
        new_btn(description="Next", on_click=partial(nav, offset=1)),
    ]))
display(output)
show_example()

Output()